In [53]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import pandas as pd
from tqdm import tqdm  # Per la barra di caricamento
import platform
from torchvision.models import AlexNet_Weights, GoogLeNet_Weights
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import csv


In [54]:
def test_model(model_path, type, average, num_classes=2):
    # Percorso al CSV (puoi definirlo qui)
    csv_path = 'final_dataset/test/df_paths_test.csv'

    # Funzione per caricare il modello
    def load_model(model_path, device):
        if type == 'googlenet':
            model = models.googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)
            model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modifica il classificatore finale
            model.aux1 = None  # Disabilita le teste ausiliarie
            model.aux2 = None
        elif type == 'alexnet':
            model = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
            model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)  # Modifica il classificatore finale
        
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        model.eval()
        return model

    # Funzione per caricare un'immagine e applicare le trasformazioni
    def process_image(image_path):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)
        return image
    
    def calculate_metrics(labels, outputs):
        # Convert labels to binary
        labels_binary = [1 if label == 'Target' else 0 for label in labels]
        # Convert outputs to binary
        outputs_binary = [1 if output == 'Target' else 0 for output in outputs]
        
        accuracy = accuracy_score(labels_binary, outputs_binary)
        precision = precision_score(labels_binary, outputs_binary, average=average)
        recall = recall_score(labels_binary, outputs_binary, average=average)
        f1 = f1_score(labels_binary, outputs_binary, average=average)
        return accuracy, precision, recall, f1
    
    def save_metrics(accuracy, precision, recall, f1, path):
        with open(path, 'w', newline='') as csvfile:
            fieldnames = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow({
                'test_accuracy': test_accuracy,
                'test_precision': test_precision,
                'test_recall': test_recall,
                'test_f1': test_f1
            })

    # Carica il dispositivo (GPU se disponibile, altrimenti CPU)
    if platform.system() == 'Windows':
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    # Carica il modello
    model = load_model(model_path, device)

    # Leggi il CSV
    df = pd.read_csv(csv_path)

    # Inizializza le liste per le predizioni e i target
    predictions = []
    targets = []

    # Itera attraverso i dati di test e fai previsioni
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Testing"):
        image_path = row['FilePath']  # Assicurati che il CSV abbia una colonna 'FilePath'
        image = process_image(image_path).to(device)

        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
            predicted_label = 'Target' if predicted.item() == 1 else 'Non-Target'  # Mappa l'output del modello alle etichette nel CSV

        predictions.append(predicted_label)
        targets.append(row['Label'])  # Assicurati che il CSV abbia una colonna 'Label' con 'Target' e 'Non-Target'

    test_accuracy, test_precision, test_recall, test_f1 = calculate_metrics(predictions, targets)
    
    name = model_path.split('/')[1]
    save_metrics(test_accuracy, test_precision, test_recall, test_f1, f'test_results/{name}.csv')

    # Stampa i risultati
    print(f'Accuracy: {test_accuracy:.4f} - Precision: {test_precision:.4f} - Recall: {test_recall:.4f} - F1: {test_f1:.4f}')

## Test con GoogLeNet

In [55]:
test_model('models/test1_google_net/best_model.pt', 'googlenet', 'binary')

Testing: 100%|██████████| 7689/7689 [04:11<00:00, 30.52it/s]

Accuracy: 0.9952 - Precision: 0.9973 - Recall: 0.9974 - F1: 0.9974


In [56]:
test_model('models/test2_google_net/best_model.pt', 'googlenet', 'weighted')

Testing: 100%|██████████| 7689/7689 [03:46<00:00, 34.00it/s]

Accuracy: 0.9945 - Precision: 0.9946 - Recall: 0.9945 - F1: 0.9945


In [57]:
test_model('models/test3_google_net/best_model.pt', 'googlenet', 'macro')

Testing: 100%|██████████| 7689/7689 [03:46<00:00, 33.90it/s]

Accuracy: 0.9960 - Precision: 0.9899 - Recall: 0.9836 - F1: 0.9867


In [58]:
test_model('models/test4_google_net/best_model.pt', 'googlenet', 'micro')

Testing: 100%|██████████| 7689/7689 [03:56<00:00, 32.53it/s]

Accuracy: 0.9949 - Precision: 0.9949 - Recall: 0.9949 - F1: 0.9949


## Test con AlexNet

In [59]:
test_model('models/test5_alex_net/checkpoint_epoch_8.pt', 'alexnet', 'weighted')

Testing: 100%|██████████| 7689/7689 [02:24<00:00, 53.25it/s]


Accuracy: 0.9625 - Precision: 0.9755 - Recall: 0.9625 - F1: 0.9666
